`ApJdataFrames` 008: Luhman2012
---
`Title`: THE DISK POPULATION OF THE UPPER SCORPIUS ASSOCIATION  
`Authors`: K. L. Luhman and E. E. Mamajek  

Data is from this paper:  
http://iopscience.iop.org/0004-637X/758/1/31/article#apj443828t1

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from astropy.io import ascii

In [3]:
import pandas as pd

## Table 1 - VOTable with all source properties

In [4]:
tbl1 = ascii.read("http://iopscience.iop.org/0004-637X/758/1/31/suppdata/apj443828t1_mrt.txt")

In [5]:
tbl1.columns

<TableColumns names=('Name','OtherNames','SpType','r_SpType','Adopt','3.6mag','e_3.6mag','f_3.6mag','4.5mag','e_4.5mag','f_4.5mag','5.8mag','e_5.8mag','f_5.8mag','8.0mag','e_8.0mag','f_8.0mag','24mag','e_24mag','f_24mag','W1mag','e_W1mag','f_W1mag','W2mag','e_W2mag','f_W2mag','W3mag','e_W3mag','f_W3mag','W4mag','e_W4mag','f_W4mag','Exc4.5','Exc8.0','Exc24','ExcW2','ExcW3','ExcW4','DiskType')>

In [6]:
tbl1[0:5]

Name,OtherNames,SpType,r_SpType,Adopt,3.6mag,e_3.6mag,f_3.6mag,4.5mag,e_4.5mag,f_4.5mag,5.8mag,e_5.8mag,f_5.8mag,8.0mag,e_8.0mag,f_8.0mag,24mag,e_24mag,f_24mag,W1mag,e_W1mag,f_W1mag,W2mag,e_W2mag,f_W2mag,W3mag,e_W3mag,f_W3mag,W4mag,e_W4mag,f_W4mag,Exc4.5,Exc8.0,Exc24,ExcW2,ExcW3,ExcW4,DiskType
,,,,,mag,mag,,mag,mag,,mag,mag,,mag,mag,,mag,mag,,mag,mag,,mag,mag,,mag,mag,,mag,mag,,,,,,,,
string224,string344,string184,string104,string64,float64,float64,string40,float64,float64,string40,float64,float64,string40,float64,float64,string40,float64,float64,string40,float64,float64,string40,float64,float64,string40,float64,float64,string40,float64,float64,string40,string8,string8,string16,string16,string16,string16,string128
2MASS J15321033-2158004,"HD 138343,HIP 76071",B9V,1,B9V,--,--,out,7.05,0.02,--,--,--,out,7.12,0.03,--,7.09,0.04,--,7.02,0.03,--,7.06,0.02,--,7.09,0.02,--,7.13,0.11,--,N,N,N,N,N,N,--
2MASS J15350863-2532397,--,M5.5,2,M5.5,--,--,out,--,--,out,--,--,out,--,--,out,--,--,out,13.37,0.03,--,13.14,0.03,--,--,--,false,--,--,nodet,--,--,--,N,--,--,--
2MASS J15351610-2544030,"HD 138813,HIP 76310",A0V,1,A0V,--,--,out,7.15,0.02,--,--,--,out,7.16,0.03,--,4.08,0.04,--,7.14,0.03,--,7.18,0.02,--,6.72,0.02,--,4.23,0.02,--,N,N,Y,N,Y,Y,debris/ev trans
2MASS J15354856-2958551,RX J1535.8-2958,M4,3,M4,--,--,out,--,--,out,--,--,out,--,--,out,--,--,out,9.18,0.02,--,8.84,0.02,--,7.09,0.02,--,5.15,0.03,--,--,--,--,Y,Y,Y,full
2MASS J15355111-2021008,--,M8,2,M8,--,--,out,--,--,out,--,--,out,--,--,out,--,--,out,13.45,0.03,--,13.08,0.04,--,--,--,nodet,--,--,nodet,--,--,--,N,--,--,--


In [7]:
len(tbl1)

863

#Cross match with SIMBAD

In [8]:
from astroquery.simbad import Simbad
import astropy.coordinates as coord
import astropy.units as u

In [9]:
customSimbad = Simbad()
customSimbad.add_votable_fields('otype', 'sptype')

In [10]:
query_list = tbl1["Name"].data.data
result = customSimbad.query_objects(query_list, verbose=True)

In [11]:
result[0:3]

MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE,OTYPE,SP_TYPE,SP_QUAL,SP_NATURE,SP_BIBCODE
,"""h:m:s""","""d:m:s""",,,mas,mas,deg,,,,,,,,
object,string104,string104,int16,int16,float32,float32,int16,string8,string8,object,object,object,string8,string8,object
HD 138343,15 32 10.3328,-21 58 00.560,9,9,7.730,5.340,90,A,O,2007A&A...474..653V,Star,B9V,C,,1988MSS...C04....0H
2MASS J15350863-2532397,15 35 08.640,-25 32 39.73,7,7,60.000,60.000,45,B,N,2003yCat.2246....0C,Star,,,,
HD 138813,15 35 16.1054,-25 44 03.006,9,9,5.360,4.470,90,A,O,2007A&A...474..653V,Star,A0V,C,,1988MSS...C04....0H


In [12]:
print "There were {} sources queried, and {} sources found.".format(len(query_list), len(result))
if len(query_list) == len(result):
    print "Hooray!  Everything matched"
else:
    print "Which ones were not found?"

There were 863 sources queried, and 862 sources found.
Which ones were not found?


In [13]:
def add_input_column_to_simbad_result(self, input_list, verbose=False):
    """
    Adds 'INPUT' column to the result of a Simbad query

    Parameters
    ----------
    object_names : sequence of strs
            names of objects from most recent query
    verbose : boolean, optional
        When `True`, verbose output is printed

    Returns
    -------
    table : `~astropy.table.Table`
        Query results table
    """
    error_string = self.last_parsed_result.error_raw
    fails = []

    for error in error_string.split("\n"):
        start_loc = error.rfind(":")+2
        fail = error[start_loc:]
        fails.append(fail)

    successes = [s for s in input_list if s not in fails]
    if verbose:
        out_message = "There were {} successful Simbad matches and {} failures."
        print out_message.format(len(successes), len(fails))

    self.last_parsed_result.table["INPUT"] = successes

    return self.last_parsed_result.table

In [14]:
result_fix = add_input_column_to_simbad_result(customSimbad, query_list, verbose=True)

There were 862 successful Simbad matches and 1 failures.
Failures are:  [u'Oph J1622-2405 B']


In [17]:
tbl1_pd = tbl1.to_pandas()
result_pd = result_fix.to_pandas()
tbl1_plusSimbad = pd.merge(tbl1_pd, result_pd, how="left", left_on="Name", right_on="INPUT")

# Save the data table locally.

In [19]:
tbl1_plusSimbad.head()

,Name,OtherNames,SpType,r_SpType,Adopt,3.6mag,e_3.6mag,f_3.6mag,4.5mag,e_4.5mag,...,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE,OTYPE,SP_TYPE,SP_QUAL,SP_NATURE,SP_BIBCODE,INPUT
0,2MASS J15321033-2158004,"HD 138343,HIP 76071",B9V,1,B9V,NaN,NaN,out,7.05,0.02,...,90,A,O,2007A&A...474..653V,Star,B9V,C,,1988MSS...C04....0H,2MASS J15321033-2158004
1,2MASS J15350863-2532397,NaN,M5.5,2,M5.5,NaN,NaN,out,NaN,NaN,...,45,B,N,2003yCat.2246....0C,Star,,,,,2MASS J15350863-2532397
2,2MASS J15351610-2544030,"HD 138813,HIP 76310",A0V,1,A0V,NaN,NaN,out,7.15,0.02,...,90,A,O,2007A&A...474..653V,Star,A0V,C,,1988MSS...C04....0H,2MASS J15351610-2544030
3,2MASS J15354856-2958551,RX J1535.8-2958,M4,3,M4,NaN,NaN,out,NaN,NaN,...,90,B,I,2003yCat.2246....0C,**,M4,D,,2000A&A...356..541K,2MASS J15354856-2958551
4,2MASS J15355111-2021008,NaN,M8,2,M8,NaN,NaN,out,NaN,NaN,...,17,B,N,2003yCat.2246....0C,Star,,,,,2MASS J15355111-2021008


In [20]:
! mkdir ../data/Luhman2012/

mkdir: ../data/Luhman2012/: File exists


In [22]:
tbl1_plusSimbad.to_csv("../data/Luhman2012/tbl1_plusSimbad.csv", index=False)

*The end*